In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sklearn
import numpy as np
import pandas as pd

In [ ]:
# data
data = pd.read_csv("/content/sample_data/100k.csv", error_bad_lines=False)

# # test data
# test = pd.read_csv("/content/test.csv")

<ipython-input-7-d92847e7c865>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("/content/sample_data/100k.csv", error_bad_lines=False)


In [ ]:
data.head()

,Unnamed: 0,tweets,labels
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1,"Try talking with ChatGPT, our new AI system wh...",good
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3,"THRILLED to share that ChatGPT, our new model ...",good
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad


In [ ]:
data.tail()

,Unnamed: 0,tweets,labels
99994,99994,ChatGPT is a development on par with the print...,bad
99995,99995,"if you're creating short form video content, a...",good
99996,99996,(@)sonofmosiah:\nDid OpenAI really market Chat...,good
99997,99997,How to Code Web 3.0 Apps with ChatGPT A.I. #Ch...,bad
99998,99998,"Actually, you can serve ads profitably off of ...",good


In [ ]:

sum(data["labels"] == "good")

26139

In [ ]:
sum(data["labels"] == "bad")

48914

In [ ]:
sum(data["labels"] == "neutral")

24946

In [ ]:
data.isnull().sum()

Unnamed: 0    0
tweets        0
labels        0
dtype: int64

#DATA PROCESSING


In [ ]:
#install tweet-preprocessor to clean tweets
!pip install tweet-preprocessor


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|()|()|(\%)|($)|(\>)|(\<)|(\{)|(\})")

REPLACE_WITH_SPACE = re.compile("(<br\\s><br\\s?)|(-)|()|(:).\)")

In [ ]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub("", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [ ]:
# clean training data
data_tweet = clean_tweets(data["tweets"])
data_tweet = pd.DataFrame(data_tweet)

In [ ]:
# append cleaned tweets to the training data
data["clean_tweet"] = data_tweet

# compare the cleaned and uncleaned tweets
data.head(10)

,Unnamed: 0,tweets,labels,clean_tweet
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral,chatgpt optimizing language models for dialogue
1,1,"Try talking with ChatGPT, our new AI system wh...",good,try talking with chatgpt our new ai system whi...
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral,chatgpt optimizing language models for dialogu...
3,3,"THRILLED to share that ChatGPT, our new model ...",good,thrilled to share that chatgpt our new model o...
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad,as of minutes ago released their new chatgpt \...
5,5,"Just launched ChatGPT, our new AI system which...",good,just launched chatgpt our new ai system which ...
6,6,"As of 2 minutes ago, @OpenAI released their ne...",bad,as of minutes ago released their new chatgpt \...
7,7,ChatGPT coming out strong refusing to help me ...,good,chatgpt coming out strong refusing to help me ...
8,8,#0penAl just deployed a thing I've been helpin...,good,just deployed a thing ive been helping build t...
9,9,Research preview of our newest model: ChatGPT\...,neutral,research preview of our newest model chatgpt\n...


In [ ]:
data["clean_tweet"]

0          chatgpt optimizing language models for dialogue
1        try talking with chatgpt our new ai system whi...
2        chatgpt optimizing language models for dialogu...
3        thrilled to share that chatgpt our new model o...
4        as of minutes ago released their new chatgpt \...
                               ...                        
99994    chatgpt is a development on par with the print...
99995    if youre creating short form video content ask...
99996    ()sonofmosiahndid openai really market chatgpt...
99997              how to code web apps with chatgpt ai \n
99998    actually you can serve ads profitably off of a...
Name: clean_tweet, Length: 99999, dtype: object

#Split Train and Test Data


In [ ]:

from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = data.labels.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(data.clean_tweet.values, y,
                                                    stratify=y,
                                                    random_state=1,
                                                    test_size=0.2, shuffle=True)


In [ ]:
x_train

array(['chatgpt can give somewhat succinct answers pretty decent asymmetry this prompt was prompted by twelve monkeys and the russian adoption of the gregorian calendar in ',
       'last weekend released  a that interacts with humans in a way \\n\\nthe result users of the tool got really impressed and social media went crazy \\n\\n1/6',
       'chatgpt the best ai chatbot ever released to the general public inspires awe and concern',
       ..., 'chatgpt could be revolutionary this shit is insane',
       'interview questions in a remote setting\\n\\nwell difficult with chatgpt',
       's company openais becomes the fastest to touch million users in just days\\n'],
      dtype=object)

#Vectorize tweets using CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
documents = ["This is Import Data's Youtube channel",
             "Data science is my passion and it is fun!",
             "Please subscribe to my channel"]

# initializing the countvectorizer
vectorizer = CountVectorizer()

# tokenize and make the document into a matrix
document_term_matrix = vectorizer.fit_transform(documents)

# check the result
pd.DataFrame(document_term_matrix.toarray(), columns = vectorizer.get_feature_names_out())

,and,channel,data,fun,import,is,it,my,passion,please,science,subscribe,this,to,youtube
0,0,1,1,0,1,1,0,0,0,0,0,0,1,0,1
1,1,0,1,1,0,2,1,1,1,0,1,0,0,0,0
2,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)


#MODEL BUILDING


In [ ]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

#Accuracy score for SVC

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')